In [1]:
import pandas as pd

from X5_ner_MiLky_way.models.naiv_replace.replace_metod import annotation
from sqlalchemy.dialects.mssql.information_schema import columns

In [2]:
# ТУТ замени
train_path = '/Users/egorbykov/Desktop/Работа/2025/hackatons/mos/X5_ner_MiLky_way/data/Датасет/train.csv'

data = pd.read_csv(train_path, sep = ';')

In [4]:
data

,sample,annotation
0,aa,"[(0, 2, 'O')]"
1,aala,"[(0, 4, 'O')]"
2,aarcca,"[(0, 6, 'O')]"
3,abon,"[(0, 4, 'O')]"
4,abso,"[(0, 4, 'B-BRAND')]"
...,...,...
27246,№1 кофейни,"[(0, 2, 'B-BRAND'), (3, 10, 'B-TYPE')]"
27247,№1 кофейник,"[(0, 2, 'B-BRAND'), (3, 11, 'B-TYPE')]"
27248,№1 пиво,"[(0, 2, 'B-BRAND'), (3, 7, 'B-TYPE')]"
27249,№1 са,"[(0, 2, 'B-BRAND'), (3, 5, 'B-TYPE')]"


In [8]:
import sys
sys.path.append('../src')

In [19]:
from text_augmentation import augment_text, add_typos, add_truncation, add_mid_deletion

In [18]:
data['text_with_typos'] = data['sample'].apply(lambda x: add_typos(x, typo_prob=0.15, max_typos_per_word=2))

In [21]:
data['text_with_truncation'] = data['sample'].apply(lambda x: add_truncation(x, trunc_prob=0.15, max_truncated_chars = 2))

In [22]:
data['text_mid_deletion'] = data['sample'].apply(lambda x: add_mid_deletion(x, deletion_prob=0.15, max_deletions_per_word = 2))

In [29]:
data

,sample,annotation,text_with_typos,text_with_truncation,text_mid_deletion
0,aa,"[(0, 2, 'O')]",aa,aa,aa
1,aala,"[(0, 4, 'O')]",aala,aala,aala
2,aarcca,"[(0, 6, 'O')]",aarcca,aarcca,arca
3,abon,"[(0, 4, 'O')]",abon,abon,aon
4,abso,"[(0, 4, 'B-BRAND')]",abso,abso,abo
...,...,...,...,...,...
27246,№1 кофейни,"[(0, 2, 'B-BRAND'), (3, 10, 'B-TYPE')]",№1 кофейни,№1 кофейни,№1 кфейи
27247,№1 кофейник,"[(0, 2, 'B-BRAND'), (3, 11, 'B-TYPE')]",№1 клфейник,№1 кофейник,№1 кфейик
27248,№1 пиво,"[(0, 2, 'B-BRAND'), (3, 7, 'B-TYPE')]",№1 пьво,№1 пиво,№1 пво
27249,№1 са,"[(0, 2, 'B-BRAND'), (3, 5, 'B-TYPE')]",№1 са,№1 са,№1 са


In [32]:
sample = pd.concat([data['sample'], data['text_with_typos'], data['text_with_truncation'], data['text_mid_deletion']], axis = 0)

In [33]:
annotation = pd.concat([data['annotation'], data['annotation'], data['annotation'], data['annotation']], axis = 0)

In [34]:
data.columns

Index(['sample', 'annotation', 'text_with_typos', 'text_with_truncation',
       'text_mid_deletion'],
      dtype='object')

In [42]:
new_dataset = pd.concat([sample, annotation], axis = 1)

In [44]:
new_dataset.columns =  ['sample', 'annotation']

In [46]:
new_dataset = new_dataset.drop_duplicates()

In [49]:
new_dataset.to_csv('../data/Датасет/train_augmentayion.csv', index = False, sep = ';')